In [79]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from collections import deque
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
import plotly.io as pio
pio.renderers.default = "browser"

In [2]:
trades = pd.read_csv('AugSept copy.csv')

In [3]:
trades.head(10)

,Transfer CCY,Transfer Date,Order ID,Buy/Sell,Instrument type,Instrument subtype,Instrument,Instrument CCY,Trade Amount,Trade Price,Settled PL
0,USD,8/6/25 15:57,705174,Sell,FOREX,Forex Majors,EUR/USD,USD,"50,000.00",1.16465,—
1,USD,8/6/25 15:58,705180,Buy,FOREX,Forex Majors,EUR/USD,USD,"50,000.00",1.16453,6
2,USD,8/6/25 15:59,705188,Sell,FOREX,Forex Majors,EUR/USD,USD,"20,000.00",1.16462,—
3,USD,8/13/25 9:22,723803,Sell,FOREX,Forex Majors,EUR/USD,USD,"100,000.00",1.17259,—
4,USD,8/13/25 9:22,723810,Sell,FOREX,Forex Majors,EUR/USD,USD,"260,000.00",1.17259,—
5,USD,8/13/25 9:22,723817,Sell,FOREX,Forex Majors,EUR/USD,USD,"100,000.00",1.17259,—
6,USD,8/13/25 9:22,723824,Sell,FOREX,Forex Majors,EUR/USD,USD,"260,000.00",1.17259,—
7,USD,8/13/25 9:22,723831,Sell,FOREX,Forex Majors,EUR/USD,USD,"100,000.00",1.17259,—
8,USD,8/13/25 9:22,723838,Sell,FOREX,Forex Majors,EUR/USD,USD,"20,000.00",1.17259,—
9,USD,8/13/25 9:27,723857,Sell,FOREX,Forex Majors,EUR/USD,USD,"100,000.00",1.17295,—


In [4]:
trades.nunique()

Transfer CCY              1
Transfer Date          1970
Order ID              11267
Buy/Sell                  2
Instrument type           2
Instrument subtype        2
Instrument                3
Instrument CCY            1
Trade Amount            169
Trade Price            5141
Settled PL             4433
dtype: int64

In [5]:
trades.isnull().sum()

Transfer CCY          0
Transfer Date         0
Order ID              0
Buy/Sell              0
Instrument type       0
Instrument subtype    0
Instrument            0
Instrument CCY        0
Trade Amount          0
Trade Price           0
Settled PL            0
dtype: int64

In [6]:
trades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11267 entries, 0 to 11266
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Transfer CCY        11267 non-null  object 
 1   Transfer Date       11267 non-null  object 
 2   Order ID            11267 non-null  int64  
 3   Buy/Sell            11267 non-null  object 
 4   Instrument type     11267 non-null  object 
 5   Instrument subtype  11267 non-null  object 
 6   Instrument          11267 non-null  object 
 7   Instrument CCY      11267 non-null  object 
 8   Trade Amount        11267 non-null  object 
 9   Trade Price         11267 non-null  float64
 10  Settled PL          11267 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 968.4+ KB


In [7]:
trades.shape

(11267, 11)

<p>NB: We have no missing values for this dataset but we have (-) for the settled PL</p>

In [8]:
# trades['Instrument CCY'].unique()
print(f"Buy/Sell values: {trades['Buy/Sell'].unique()}")
print(f"Instruments: {trades['Instrument'].unique()}")
print(f"Instrument Types: {trades['Instrument type'].unique()}")
print(f"Instrument Subtypes: {trades['Instrument subtype'].unique()}")

Buy/Sell values: ['Sell' 'Buy']
Instruments: ['EUR/USD' 'XAU/USD' 'GBP/USD']
Instrument Types: ['FOREX' 'CFD']
Instrument Subtypes: ['Forex Majors' 'Metals']


<p> Date time processing </p>

In [9]:
trades.head()

,Transfer CCY,Transfer Date,Order ID,Buy/Sell,Instrument type,Instrument subtype,Instrument,Instrument CCY,Trade Amount,Trade Price,Settled PL
0,USD,8/6/25 15:57,705174,Sell,FOREX,Forex Majors,EUR/USD,USD,"50,000.00",1.16465,—
1,USD,8/6/25 15:58,705180,Buy,FOREX,Forex Majors,EUR/USD,USD,"50,000.00",1.16453,6
2,USD,8/6/25 15:59,705188,Sell,FOREX,Forex Majors,EUR/USD,USD,"20,000.00",1.16462,—
3,USD,8/13/25 9:22,723803,Sell,FOREX,Forex Majors,EUR/USD,USD,"100,000.00",1.17259,—
4,USD,8/13/25 9:22,723810,Sell,FOREX,Forex Majors,EUR/USD,USD,"260,000.00",1.17259,—


In [10]:
time_trades = trades.copy()
time_trades['Transfer Date'] = pd.to_datetime(time_trades['Transfer Date'], format='%m/%d/%y %H:%M')
# Finding the trading period for this dataset
period = time_trades['Transfer Date'].max() - time_trades['Transfer Date'].min()
print(f'Trading period: {period}')
# Understanding range of dates for trading
print(f"Trading date ranges: {time_trades['Transfer Date'].min()} to {time_trades['Transfer Date'].max()}")
# Change the dates to ubderstand the day of week trading activity
time_trades['Trading Day'] = time_trades['Transfer Date'].dt.date
time_trades['Trading Time'] = time_trades['Transfer Date'].dt.time
time_trades['Trading Hour'] = time_trades['Transfer Date'].dt.hour
time_trades['Day of Week'] = time_trades['Transfer Date'].dt.day_name()
time_trades['Trading Week'] = time_trades['Transfer Date'].dt.isocalendar().week
time_trades['Trading Month'] = time_trades['Transfer Date'].dt.month
time_trades['Calendar Month'] = time_trades['Transfer Date'].dt.month_name()

# Let us remove the (,) in the Trade Amount
time_trades['Trade Amount'] = time_trades['Trade Amount'].str.replace(',', '').astype(float)

# NB: We have no missing values for this dataset but we have (-) for the settled PL so we will assume (-) is for the open slots
time_trades['Settled PL Initial'] = time_trades['Settled PL']
time_trades['Settled PL'] = time_trades['Settled PL'].replace('-', np.nan)
time_trades['Settled PL'] = time_trades['Settled PL'].astype(str).str.replace(',','')
time_trades['Settled PL'] = pd.to_numeric(time_trades['Settled PL'], errors='coerce')

print(f"Buy/Sell values: {time_trades['Buy/Sell'].unique()}")
print(f"Instruments: {time_trades['Instrument'].unique()}")
print(f"Instrument Types: {time_trades['Instrument type'].unique()}")
print(f"Instrument Subtypes: {time_trades['Instrument subtype'].unique()}")


Trading period: 54 days 17:52:00
Trading date ranges: 2025-08-06 15:57:00 to 2025-09-30 09:49:00
Buy/Sell values: ['Sell' 'Buy']
Instruments: ['EUR/USD' 'XAU/USD' 'GBP/USD']
Instrument Types: ['FOREX' 'CFD']
Instrument Subtypes: ['Forex Majors' 'Metals']


In [11]:
time_trades.head()

,Transfer CCY,Transfer Date,Order ID,Buy/Sell,Instrument type,Instrument subtype,Instrument,Instrument CCY,Trade Amount,Trade Price,Settled PL,Trading Day,Trading Time,Trading Hour,Day of Week,Trading Week,Trading Month,Calendar Month,Settled PL Initial
0,USD,2025-08-06 15:57:00,705174,Sell,FOREX,Forex Majors,EUR/USD,USD,50000.0,1.16465,NaN,2025-08-06,15:57:00,15,Wednesday,32,8,August,—
1,USD,2025-08-06 15:58:00,705180,Buy,FOREX,Forex Majors,EUR/USD,USD,50000.0,1.16453,6.0,2025-08-06,15:58:00,15,Wednesday,32,8,August,6
2,USD,2025-08-06 15:59:00,705188,Sell,FOREX,Forex Majors,EUR/USD,USD,20000.0,1.16462,NaN,2025-08-06,15:59:00,15,Wednesday,32,8,August,—
3,USD,2025-08-13 09:22:00,723803,Sell,FOREX,Forex Majors,EUR/USD,USD,100000.0,1.17259,NaN,2025-08-13,09:22:00,9,Wednesday,33,8,August,—
4,USD,2025-08-13 09:22:00,723810,Sell,FOREX,Forex Majors,EUR/USD,USD,260000.0,1.17259,NaN,2025-08-13,09:22:00,9,Wednesday,33,8,August,—


In [12]:
# Calculating the different trade fields to use
# Direction of trade
time_trades['Trade Direction'] = time_trades['Buy/Sell'].map({'Buy': 1, 'Sell': -1})
time_trades['Signed Trade Amount'] = time_trades['Trade Amount'] * time_trades['Trade Direction']
# Settled PL position
time_trades['Is_settled'] = time_trades['Settled PL'].notna()
settled_count = time_trades['Is_settled'].sum()
unsettled_count = (~time_trades['Is_settled']).sum()
print(f"Number of settled trades: {settled_count}")
print(f"Number of unsettled trades: {unsettled_count}")
print(f"Settled trades: {settled_count:,} ({settled_count/len(time_trades)*100:.2f}%)")
print(f"Unsettled trades: {unsettled_count:,} ({unsettled_count/len(time_trades)*100:.2f}%)")
# Grouping per instruments
time_trades['Instrument Type'] = time_trades['Instrument type']
time_trades['Instrument Category'] = time_trades['Instrument subtype']

Number of settled trades: 5649
Number of unsettled trades: 5618
Settled trades: 5,649 (50.14%)
Unsettled trades: 5,618 (49.86%)


In [13]:
time_trades.head()

,Transfer CCY,Transfer Date,Order ID,Buy/Sell,Instrument type,Instrument subtype,Instrument,Instrument CCY,Trade Amount,Trade Price,...,Day of Week,Trading Week,Trading Month,Calendar Month,Settled PL Initial,Trade Direction,Signed Trade Amount,Is_settled,Instrument Type,Instrument Category
0,USD,2025-08-06 15:57:00,705174,Sell,FOREX,Forex Majors,EUR/USD,USD,50000.0,1.16465,...,Wednesday,32,8,August,—,-1,-50000.0,False,FOREX,Forex Majors
1,USD,2025-08-06 15:58:00,705180,Buy,FOREX,Forex Majors,EUR/USD,USD,50000.0,1.16453,...,Wednesday,32,8,August,6,1,50000.0,True,FOREX,Forex Majors
2,USD,2025-08-06 15:59:00,705188,Sell,FOREX,Forex Majors,EUR/USD,USD,20000.0,1.16462,...,Wednesday,32,8,August,—,-1,-20000.0,False,FOREX,Forex Majors
3,USD,2025-08-13 09:22:00,723803,Sell,FOREX,Forex Majors,EUR/USD,USD,100000.0,1.17259,...,Wednesday,33,8,August,—,-1,-100000.0,False,FOREX,Forex Majors
4,USD,2025-08-13 09:22:00,723810,Sell,FOREX,Forex Majors,EUR/USD,USD,260000.0,1.17259,...,Wednesday,33,8,August,—,-1,-260000.0,False,FOREX,Forex Majors


In [15]:
# Sorting the trades by date
time_trades = time_trades.sort_values(by='Transfer Date').reset_index(drop=True)
# Checking if we have duplicate orders for the order IDs
duplicate_orders = time_trades.duplicated(subset=['Order ID']).sum()
if duplicate_orders > 0:
    print(f"Found {duplicate_orders} duplicate Order IDs.")
else:
    print("No duplicate Order IDs found.")

# Let us check if we have simultaneous trades
simultaneous_trades = time_trades.groupby('Transfer Date').size()
max_simultaneous = simultaneous_trades.max()
print(f"Maximum simultaneous trades at the same timestamp: {max_simultaneous}")
if max_simultaneous > 1:
    print("There are simultaneous trades at the same timestamp.")

No duplicate Order IDs found.
Maximum simultaneous trades at the same timestamp: 117
There are simultaneous trades at the same timestamp.


<p>NB: we have simultaneous trades (117)</p>

In [20]:
# Segmenting the trades by instrument type and product type
# Instruments: ['EUR/USD' 'XAU/USD' 'GBP/USD']
# Instrument Types: ['FOREX' 'CFD']
# Instrument Subtypes: ['Forex Majors' 'Metals']
trade_eur = time_trades[time_trades['Instrument'] == 'EUR/USD'].copy().reset_index(drop=True)
trade_xau = time_trades[time_trades['Instrument'] == 'XAU/USD'].copy().reset_index(drop=True)
trade_gbp = time_trades[time_trades['Instrument'] == 'GBP/USD'].copy().reset_index(drop=True)

print(f"EUR/USD(Euro) trades: {len(trade_eur)}")
print(f"XAU/USD(Gold) trades: {len(trade_xau)}")
print(f"GBP/USD trades: {len(trade_gbp)}")

# product type
trade_forex = time_trades[time_trades['Instrument Type'] == 'FOREX'].copy()
trade_cfd = time_trades[time_trades['Instrument Type'] == 'CFD'].copy()
print(f"\nFOREX trades: {len(trade_forex)}")
print(f"CFD trades: {len(trade_cfd)}")

# print(f"\nFOREX trades: {len(trade_eur) + len(trade_gbp)}")
# print(f"CFD trades: {len(trade_xau)}")

EUR/USD(Euro) trades: 4513
XAU/USD(Gold) trades: 6753
GBP/USD trades: 1

FOREX trades: 4514
CFD trades: 6753

FOREX trades: 4514
CFD trades: 6753


<h3> Cumulative Position over the length of trading period (Week, month, overall)

In [23]:
def cumulative_position(time_instrument, instrument_name):
    time_instrument['Cumulative Position'] = time_instrument['Signed Trade Amount'].cumsum()
    final_position = time_instrument['Cumulative Position'].iloc[-1]
    max_long = time_instrument['Cumulative Position'].max()
    max_short = time_instrument['Cumulative Position'].min()
    print(f"{instrument_name} - Final Position: {final_position}, Max Long: {max_long}, Max Short: {max_short}")
    print(f"Position for {len(time_instrument)} trades plotted below.")
    return time_instrument
# calculating for Gold and Euro
trade_eur = cumulative_position(trade_eur, 'EUR/USD')
trade_xau = cumulative_position(trade_xau, 'XAU/USD')

# updating the main dataframe
time_trades.loc[time_trades['Instrument'] == 'EUR/USD', 'Cumulative Position'] = trade_eur['Cumulative Position'].values
time_trades.loc[time_trades['Instrument'] == 'XAU/USD', 'Cumulative Position'] = trade_xau['Cumulative Position'].values


EUR/USD - Final Position: -17536000.0, Max Long: 16979000.0, Max Short: -24419000.0
Position for 4513 trades plotted below.
XAU/USD - Final Position: -2680.0, Max Long: 19077.0, Max Short: -18585.0
Position for 6753 trades plotted below.


In [25]:
# Time period aggregations
def aggregate_positions(time_instrument, instrument_name):
    # Daily aggregation
    daily_pos = time_instrument.groupby('Trading Day').agg({'Cumulative Position':'last',
                                                            'Signed Trade Amount':'sum',
                                                          'Order ID':'count'}).rename(columns={'Order ID': 'Trade Count',
                                                                                               'Signed Trade Amount':'Daily Signed Trade Amount'})
    # weekly aggregation
    weekly_pos = time_instrument.groupby('Trading Week').agg({'Cumulative Position':'last',
                                                            'Signed Trade Amount':'sum',
                                                          'Order ID':'count'}).rename(columns={'Order ID': 'Trade Count',
                                                                                               'Signed Trade Amount':'Weekly Signed Trade Amount'})
    # monthly aggregation
    monthly_pos = time_instrument.groupby('Trading Month').agg({'Cumulative Position':'last',
                                                            'Signed Trade Amount':'sum',
                                                          'Order ID':'count'}).rename(columns={'Order ID': 'Trade Count',
                                                                                               'Signed Trade Amount':'Monthly Signed Trade Amount'})
    # long short period calculations
    daily_pos['Max Long'] = time_instrument.groupby('Trading Day')['Cumulative Position'].max()
    daily_pos['Max Short'] = time_instrument.groupby('Trading Day')['Cumulative Position'].min()
    weekly_pos['Max Long'] = time_instrument.groupby('Trading Week')['Cumulative Position'].max()
    weekly_pos['Max Short'] = time_instrument.groupby('Trading Week')['Cumulative Position'].min()
    monthly_pos['Max Long'] = time_instrument.groupby('Trading Month')['Cumulative Position'].max()
    monthly_pos['Max Short'] = time_instrument.groupby('Trading Month')['Cumulative Position'].min()

    print(f"Daily aggregation {len(daily_pos)} days")
    print(f"Weekly aggregation {len(weekly_pos)} weeks")
    print(f"Monthly aggregation {len(monthly_pos)} months")
    return daily_pos, weekly_pos, monthly_pos
# Gold&Euro aggregations
eur_daily, eur_weekly, eur_monthly = aggregate_positions(trade_eur, 'EUR/USD')
xau_daily, xau_weekly, xau_monthly = aggregate_positions(trade_xau, 'XAU/USD')


Daily aggregation 33 days
Weekly aggregation 9 weeks
Monthly aggregation 2 months
Daily aggregation 40 days
Weekly aggregation 9 weeks
Monthly aggregation 2 months


In [27]:
weekly_eur_display = eur_weekly[['Cumulative Position','Max Long','Max Short','Trade Count']].copy()
weekly_eur_display.columns = ['End Position', 'Max Long', 'Max Short', 'Trades']
weekly_eur_display.head()

,End Position,Max Long,Max Short,Trades
Trading Week,,,,
32,-20000.0,5024999.0,-50000.0,29
33,-860000.0,840000.0,-2540000.0,58
34,-4536000.0,5215000.0,-4536000.0,261
35,-2421000.0,16979000.0,-8911000.0,732
36,-9081000.0,-1321000.0,-9721000.0,154


In [29]:
weekly_xau_display = xau_weekly[['Cumulative Position','Max Long','Max Short','Trade Count']].copy()
weekly_xau_display.columns = ['End Position', 'Max Long', 'Max Short', 'Trades']
weekly_xau_display.head()

,End Position,Max Long,Max Short,Trades
Trading Week,,,,
32,-2744.0,450.0,-2744.0,212
33,0.0,4238.0,-2694.0,679
34,-2435.0,3055.0,-2600.0,483
35,-1875.0,440.0,-2660.0,458
36,-10570.0,2200.0,-10570.0,899


In [32]:
monthly_summary = pd.DataFrame({
    'EUR/USD End Position': eur_monthly['Cumulative Position'],
    'XAU/USD End Position': xau_monthly['Cumulative Position'],
    'EUR/USD Trades': eur_monthly['Trade Count'],
    'XAU/USD Trades': xau_monthly['Trade Count']
})
monthly_summary.head()

,EUR/USD End Position,XAU/USD End Position,EUR/USD Trades,XAU/USD Trades
Trading Month,,,,
8,-2421000.0,-1875.0,1080,1832
9,-17536000.0,-2680.0,3433,4921


<p> Cumulative P/L (as well, with the ability to split per day, week, month, overall) </p>

In [40]:
# Profit and Loss calculations
settled_trades = time_trades['Is_settled'].sum()
total_trades = len(time_trades)
print(f"Settled trades: {settled_trades} out of {total_trades} ({settled_trades/total_trades*100:.2f}%)")
print(f"Unsettled trades: {total_trades - settled_trades} out of {total_trades} ({(total_trades - settled_trades)/total_trades*100:.2f}%)")

# total P/L
total_settled_pl = time_trades['Settled PL'].sum()
eur_pl = trade_eur['Settled PL'].sum()
xau_pl = trade_xau['Settled PL'].sum()

print(f"Total Settled P/L: {total_settled_pl:.2f}")
print(f"EUR/USD Settled P/L: {eur_pl:.2f}")
print(f"XAU/USD Settled P/L: {xau_pl:.2f}")

# Aggregated P/L per period (EURO)
daily_eur_pl = trade_eur.groupby('Trading Day').agg({'Settled PL':'sum', 'Order ID':'count'}).rename(columns={
    'Settled PL':'Daily Settled PL',
    'Order ID':'Trade Count'}) 
daily_eur_pl['Cumulative PL'] = daily_eur_pl['Daily Settled PL'].cumsum()   

weekly_eur_pl = trade_eur.groupby('Trading Week').agg({'Settled PL':'sum', 'Order ID':'count'}).rename(columns={
    'Settled PL':'Weekly Settled PL',
    'Order ID':'Trade Count'}) 
weekly_eur_pl['Cumulative PL'] = weekly_eur_pl['Weekly Settled PL'].cumsum() 

monthly_eur_pl = trade_eur.groupby('Trading Month').agg({'Settled PL':'sum', 'Order ID':'count'}).rename(columns={
    'Settled PL':'Monthly Settled PL',
    'Order ID':'Trade Count'}) 
monthly_eur_pl['Cumulative PL'] = monthly_eur_pl['Monthly Settled PL'].cumsum()

# Xau P/L
daily_xau_pl = trade_xau.groupby('Trading Day').agg({'Settled PL':'sum', 'Order ID':'count'}).rename(columns={
    'Settled PL':'Daily Settled PL',
    'Order ID':'Trade Count'}) 
daily_xau_pl['Cumulative PL'] = daily_xau_pl['Daily Settled PL'].cumsum()

weekly_xau_pl = trade_xau.groupby('Trading Week').agg({'Settled PL':'sum', 'Order ID':'count'}).rename(columns={
    'Settled PL':'Weekly Settled PL',
    'Order ID':'Trade Count'}) 
weekly_xau_pl['Cumulative PL'] = weekly_xau_pl['Weekly Settled PL'].cumsum()

monthly_xau_pl = trade_xau.groupby('Trading Month').agg({'Settled PL':'sum', 'Order ID':'count'}).rename(columns={
    'Settled PL':'Monthly Settled PL',
    'Order ID':'Trade Count'}) 
monthly_xau_pl['Cumulative PL'] = monthly_xau_pl['Monthly Settled PL'].cumsum()

trade_eur['Cumulative_PL'] = trade_eur['Settled PL'].fillna(0).cumsum()
trade_xau['Cumulative_PL'] = trade_xau['Settled PL'].fillna(0).cumsum()

# Add to main dataframe
time_trades.loc[time_trades['Instrument'] == 'EUR/USD', 'Cumulative_PL'] = trade_eur['Cumulative_PL'].values
time_trades.loc[time_trades['Instrument'] == 'XAU/USD', 'Cumulative_PL'] = trade_xau['Cumulative_PL'].values
 


Settled trades: 5649 out of 11267 (50.14%)
Unsettled trades: 5618 out of 11267 (49.86%)
Total Settled P/L: -907717.53
EUR/USD Settled P/L: 89825.28
XAU/USD Settled P/L: -997542.81


In [41]:
trade_eur.head()

,Transfer CCY,Transfer Date,Order ID,Buy/Sell,Instrument type,Instrument subtype,Instrument,Instrument CCY,Trade Amount,Trade Price,...,Trading Month,Calendar Month,Settled PL Initial,Trade Direction,Signed Trade Amount,Is_settled,Instrument Type,Instrument Category,Cumulative Position,Cumulative_PL
0,USD,2025-08-06 15:57:00,705174,Sell,FOREX,Forex Majors,EUR/USD,USD,50000.0,1.16465,...,8,August,—,-1,-50000.0,False,FOREX,Forex Majors,-50000.0,0.0
1,USD,2025-08-06 15:58:00,705180,Buy,FOREX,Forex Majors,EUR/USD,USD,50000.0,1.16453,...,8,August,6,1,50000.0,True,FOREX,Forex Majors,0.0,6.0
2,USD,2025-08-06 15:59:00,705188,Sell,FOREX,Forex Majors,EUR/USD,USD,20000.0,1.16462,...,8,August,—,-1,-20000.0,False,FOREX,Forex Majors,-20000.0,6.0
3,USD,2025-08-07 12:06:00,707131,Buy,FOREX,Forex Majors,EUR/USD,USD,1000000.0,1.16434,...,8,August,—,1,1000000.0,False,FOREX,Forex Majors,980000.0,6.0
4,USD,2025-08-07 12:11:00,707152,Sell,FOREX,Forex Majors,EUR/USD,USD,1000000.0,1.16423,...,8,August,-110,-1,-1000000.0,True,FOREX,Forex Majors,-20000.0,-104.0


In [47]:
def breakdown(time_instrument, instrument_name):
    settled = time_instrument[time_instrument['Is_settled']].copy()

    profit = settled[settled['Settled PL'] > 0]
    loss = settled[settled['Settled PL'] < 0]
    breakeven = settled[settled['Settled PL'] == 0]

    total_selected = len(settled)
    p_count = len(profit)
    l_count = len(loss)
    b_count = len(breakeven)

    p_percent = (p_count / total_selected) * 100 if total_selected > 0 else 0

    total_pl = settled['Settled PL'].sum()
    avg_pl = settled['Settled PL'].mean() 
    avg_win = profit['Settled PL'].mean() if p_count > 0 else 0
    avg_loss = loss['Settled PL'].mean() if l_count > 0 else 0
    largest_win = profit['Settled PL'].max() 
    largest_loss = loss['Settled PL'].min()
    std_deviation = settled['Settled PL'].std()

    print(f"Total settled trades for{total_selected}")
    print(f"Profitable trades: {p_count} ({p_percent:.2f}%)")
    print(f"Losing trades: {l_count:,} ({(l_count/total_selected*100):.2f}%)")
    print(f"Breakeven trades: {b_count} ({(b_count/total_selected*100):.2f}%)")
    print(f"Total P/L: {total_pl:.2f}")
    print(f"Average P/L per trade: {avg_pl:.2f}")
    print(f"Average Winning Trade: {avg_win:.2f}")
    print(f"Average Losing Trade: {avg_loss:.2f}")
    print(f"Largest Winning Trade: {largest_win:.2f}")
    print(f"Largest Losing Trade: {largest_loss:.2f}")
    print(f"Standard Deviation of P/L: {std_deviation:.2f}")

    if avg_loss != 0:
        rr_ratio = abs(avg_win / avg_loss)
        print(f"Risk-Reward Ratio: {rr_ratio:.2f}")

    return {
        'Total Settled Trades': total_selected,
        'Profitable Trade Percentage': p_percent,
        'Profitable Trades': p_count,
        'Losing Trades': l_count,
        'Breakeven Trades': b_count,
        'Total P/L': total_pl,
        'Average P/L': avg_pl,
        'Average Winning Trade': avg_win,
        'Average Losing Trade': avg_loss,
        'Largest Winning Trade': largest_win,
        'Largest Losing Trade': largest_loss    
    }
eur_breakdown = breakdown(trade_eur, 'EUR/USD')
xau_breakdown = breakdown(trade_xau, 'XAU/USD')

Total settled trades for2183
Profitable trades: 1238 (56.71%)
Losing trades: 942 (43.15%)
Breakeven trades: 3 (0.14%)
Total P/L: 89825.28
Average P/L per trade: 41.15
Average Winning Trade: 296.52
Average Losing Trade: -294.33
Largest Winning Trade: 5768.09
Largest Losing Trade: -4335.18
Standard Deviation of P/L: 536.08
Risk-Reward Ratio: 1.01
Total settled trades for3466
Profitable trades: 1686 (48.64%)
Losing trades: 1,780 (51.36%)
Breakeven trades: 0 (0.00%)
Total P/L: -997542.81
Average P/L per trade: -287.81
Average Winning Trade: 626.80
Average Losing Trade: -1154.11
Largest Winning Trade: 7414.55
Largest Losing Trade: -21394.64
Standard Deviation of P/L: 1616.41
Risk-Reward Ratio: 0.54


In [51]:
# Creating a summary dataframe

weekly_pl_summary = pd.DataFrame({
    'EUR/USD Weekly P/L': weekly_eur_pl['Cumulative PL'],
    'EUR/USD Cumulative': weekly_eur_pl['Cumulative PL'],
    'XAU/USD Weekly P/L': weekly_xau_pl['Cumulative PL'],
    'XAU/USD Cumulative': weekly_xau_pl['Cumulative PL']
    })
weekly_pl_summary['Total Weekly P/L'] = weekly_pl_summary['EUR/USD Weekly P/L'] + weekly_pl_summary['XAU/USD Weekly P/L']

monthly_pl_summary = pd.DataFrame({
    'EUR/USD Monthly P/L': monthly_eur_pl['Cumulative PL'],
    'EUR/USD Cumulative Trades': monthly_eur_pl['Cumulative PL'],
    'XAU/USD Monthly P/L': monthly_xau_pl['Cumulative PL'],
    'XAU/USD Cumulative Trades': monthly_xau_pl['Cumulative PL']
    })
monthly_pl_summary['Total Monthly P/L'] = monthly_pl_summary['EUR/USD Monthly P/L'] + monthly_pl_summary['XAU/USD Monthly P/L']
monthly_pl_summary['Total Trades'] = monthly_pl_summary['EUR/USD Cumulative Trades'] + monthly_pl_summary['XAU/USD Cumulative Trades']

overall_pl_summary = pd.DataFrame({
    'Metric': ['Total P/L', 'Percentage', 'Avg P/L per Trade', 'Avg Win', 'Avg Loss', 'Largest Win', 'Largest Loss'],
    'EUR/USD': [
        f"${eur_breakdown['Total P/L']:,.2f}",
        f"{eur_breakdown['Profitable Trade Percentage']:.2f}%",
        f"${eur_breakdown['Average P/L']:,.2f}",
        f"${eur_breakdown['Average Winning Trade']:,.2f}",
        f"${eur_breakdown['Average Losing Trade']:,.2f}",
        f"${eur_breakdown['Largest Winning Trade']:,.2f}",
        f"${eur_breakdown['Largest Losing Trade']:,.2f}"
    ],
    'XAU/USD': [
        f"${xau_breakdown['Total P/L']:,.2f}",
        f"{xau_breakdown['Profitable Trade Percentage']:.2f}%",
        f"${xau_breakdown['Average P/L']:,.2f}",
        f"${xau_breakdown['Average Winning Trade']:,.2f}",
        f"${xau_breakdown['Average Losing Trade']:,.2f}",
        f"${xau_breakdown['Largest Winning Trade']:,.2f}",
        f"${xau_breakdown['Largest Losing Trade']:,.2f}"
    ],
    'Overall': [
        f"${eur_breakdown['Total P/L'] + xau_breakdown['Total P/L']:,.2f}",
        f"{(eur_breakdown['Profitable Trades'] + xau_breakdown['Profitable Trades'])/(eur_breakdown['Total Settled Trades'] + xau_breakdown['Total Settled Trades'])*100:.2f}%",
        f"${(eur_breakdown['Total P/L'] + xau_breakdown['Total P/L'])/(eur_breakdown['Total Settled Trades'] + xau_breakdown['Total Settled Trades']):.2f}",
        '-',
        '-',
        f"${max(eur_breakdown['Largest Winning Trade'], xau_breakdown['Largest Winning Trade']):,.2f}",
        f"${min(eur_breakdown['Largest Losing Trade'], xau_breakdown['Largest Losing Trade']):,.2f}"
    ]
})
overall_pl_summary.head(10)

,Metric,EUR/USD,XAU/USD,Overall
0,Total P/L,"$89,825.28","$-997,542.81","$-907,717.53"
1,Percentage,56.71%,48.64%,51.76%
2,Avg P/L per Trade,$41.15,$-287.81,$-160.69
3,Avg Win,$296.52,$626.80,-
4,Avg Loss,$-294.33,"$-1,154.11",-
5,Largest Win,"$5,768.09","$7,414.55","$7,414.55"
6,Largest Loss,"$-4,335.18","$-21,394.64","$-21,394.64"


<p> Toxicity – check if there are any quick turns (basically client is buying and selling in small period of time, say 1 minute)</p>

In [53]:
# create queues for toxicity check using FIFO method
def fifo_trades(time_instrument, instrument_name):
    # unmatched trades queue
    open_buys = deque()
    open_sells = deque()

    matched_pairs = []

    for idx, row in time_instrument.iterrows():
        trade_type = row['Buy/Sell']
        amount = row['Trade Amount']
        price = row['Trade Price']
        timestamp = row['Transfer Date']
        order_id = row['Order ID']  
        settled_pl = row['Settled PL']

        if trade_type == 'Buy':
            if len(open_sells) > 0:
                sell_trade = open_sells.popleft()
                # match with the oldest sell 
                entry_time = sell_trade['timestamp']
                exit_time = timestamp
                holding_time = (exit_time - entry_time).total_seconds()
                # pl for short sell
                pl = (sell_trade['price'] - price)* amount
                matched_pairs.append({
                    'Entry_order_ID': sell_trade['order_id'],
                    'Exit_order_ID': order_id,
                    'Position_Type': 'Short',
                    'Entry_Time': entry_time,
                    'Exit_Time': exit_time,
                    'Holding_Time_Seconds': holding_time,
                    'Entry_Price': sell_trade['price'],
                    'Exit_Price': price,
                    'Amount': amount,
                    'Calculated_PL': pl,
                    'Settled_PL': settled_pl if pd.notna(settled_pl) else pl
                })
            else:
                open_buys.append({
                    'order_id': order_id,
                    'price': price,
                    'amount': amount,
                    'timestamp': timestamp
                })
        elif trade_type == 'Sell':
            if len(open_buys) > 0:
                buy_trade = open_buys.popleft()
                # match with the oldest buy
                entry_time = buy_trade['timestamp']
                exit_time = timestamp
                holding_time = (exit_time - entry_time).total_seconds()
                # pl for long buy
                pl = (price - buy_trade['price']) * amount
                matched_pairs.append({
                    'Entry_order_ID': buy_trade['order_id'],
                    'Exit_order_ID': order_id,
                    'Position_Type': 'Long',
                    'Entry_Time': entry_time,
                    'Exit_Time': exit_time,
                    'Holding_Time_Seconds': holding_time,
                    'Entry_Price': buy_trade['price'],
                    'Exit_Price': price,
                    'Amount': amount,
                    'Calculated_PL': pl,
                    'Settled_PL': settled_pl if pd.notna(settled_pl) else pl
                })
            else:
                open_sells.append({
                    'order_id': order_id,
                    'price': price,
                    'amount': amount,
                    'timestamp': timestamp
                })
    if len(matched_pairs) > 0:
        matched_df = pd.DataFrame(matched_pairs)
        # Convert holding time to minutes and hours
        matched_df['Holding_Time_Minutes'] = matched_df['Holding_Time_Seconds'] / 60
        matched_df['Holding_Time_Hours'] = matched_df['Holding_Time_Seconds'] / 3600
        matched_df['Holding_Time_Days'] = matched_df['Holding_Time_Seconds'] / 86400

        print(f"Matched {len(matched_df)} trade pairs for {instrument_name}.")
        print(f"Unmatched Buy trades: {len(open_buys)}")
        print(f"Unmatched Sell trades: {len(open_sells)}")
        return matched_df
    else:
        print(f"No matched trade pairs for {instrument_name}.")
        return pd.DataFrame()
    
eur_matched = fifo_trades(trade_eur, 'EUR/USD')
xau_matched = fifo_trades(trade_xau, 'XAU/USD')

Matched 2228 trade pairs for EUR/USD.
Unmatched Buy trades: 0
Unmatched Sell trades: 57
Matched 3325 trade pairs for XAU/USD.
Unmatched Buy trades: 103
Unmatched Sell trades: 0


In [54]:
# /Calculating the holding time 
TOXICITY_THRESHOLD_SECONDS = 60  # 1 minute
def check_toxicity(matched_df, instrument_name):
    if len(matched_df) == 0:
        print(f"No matched trades to check toxicity for {instrument_name}.")
        return matched_df
    matched_df['Is_Toxic'] = matched_df['Holding_Time_Seconds'] < TOXICITY_THRESHOLD_SECONDS
    toxic_count = matched_df['Is_Toxic'].sum()
    normal_count = (~matched_df['Is_Toxic']).sum()
    total_count = len(matched_df)
    toxicity_percent = (toxic_count / total_count) * 100 if total_count > 0 else 0
    print(f"   Total Matched Pairs: {total_count:,}")
    print(f"   Toxic Trades (≤1 min): {toxic_count:,} ({toxicity_percent:.2f}%)")
    print(f"   Normal Trades (>1 min): {normal_count:,} ({100-toxicity_percent:.2f}%)")

    # Breaking down to category
    matched_df['Hold_Category'] = pd.cut(
        matched_df['Holding_Time_Seconds'],
        bins=[-1, 60, 300, 900, 3600, 86400, np.inf],
        labels=['Toxic(≤1 min)', 'Ultra-short (1-5 min)', 'Short-Term (5-15 min)', 'Middle-Term (15-60 min)', 'Day (1-24 hrs)', 'Days(>24 hrs)']
    )
    category_counts = matched_df['Hold_Category'].value_counts().sort_index()
    for category, count in category_counts.items():
        percent = (count / total_count) * 100 if total_count > 0 else 0
        print(f"   {category}: {count:,} ({percent:.2f}%)")
    return matched_df
if len(eur_matched) > 0:
    print("\nEUR/USD Toxicity Analysis:")
    eur_matched = check_toxicity(eur_matched, 'EUR/USD')
if len(xau_matched) > 0:
    print("\nXAU/USD Toxicity Analysis:")
    xau_matched = check_toxicity(xau_matched, 'XAU/USD')
    



EUR/USD Toxicity Analysis:
   Total Matched Pairs: 2,228
   Toxic Trades (≤1 min): 0 (0.00%)
   Normal Trades (>1 min): 2,228 (100.00%)
   Toxic(≤1 min): 24 (1.08%)
   Ultra-short (1-5 min): 70 (3.14%)
   Short-Term (5-15 min): 216 (9.69%)
   Middle-Term (15-60 min): 175 (7.85%)
   Day (1-24 hrs): 1,227 (55.07%)
   Days(>24 hrs): 516 (23.16%)

XAU/USD Toxicity Analysis:
   Total Matched Pairs: 3,325
   Toxic Trades (≤1 min): 102 (3.07%)
   Normal Trades (>1 min): 3,223 (96.93%)
   Toxic(≤1 min): 213 (6.41%)
   Ultra-short (1-5 min): 247 (7.43%)
   Short-Term (5-15 min): 248 (7.46%)
   Middle-Term (15-60 min): 515 (15.49%)
   Day (1-24 hrs): 1,731 (52.06%)
   Days(>24 hrs): 371 (11.16%)


In [63]:
def toxicity_analysis(matched_df, instrument_name):
    if len(matched_df) == 0:
        return None
    
    toxic = matched_df[matched_df['Is_Toxic']]
    normal = matched_df[~matched_df['Is_Toxic']]
    
    # Volume analysis
    toxic_volume = toxic['Amount'].sum()
    total_volume = matched_df['Amount'].sum()
    toxic_volume_pct = (toxic_volume / total_volume * 100) if total_volume > 0 else 0
    
    # P/L analysis
    toxic_pl = toxic['Settled_PL'].sum()
    normal_pl = normal['Settled_PL'].sum()
    toxic_avg_pl = toxic['Settled_PL'].mean() if len(toxic) > 0 else 0
    normal_avg_pl = normal['Settled_PL'].mean() if len(normal) > 0 else 0
    
    # Time of day analysis
    toxic['Hour'] = toxic['Entry_Time'].dt.hour
    hourly_toxic = toxic.groupby('Hour').size()
    
    return {
        'total_matched': len(matched_df),
        'toxic_count': len(toxic),
        'toxic_pct': len(toxic) / len(matched_df) * 100,
        'toxic_volume': toxic_volume,
        'toxic_volume_pct': toxic_volume_pct,
        'toxic_pl': toxic_pl,
        'normal_pl': normal_pl,
        'toxic_avg_pl': toxic_avg_pl,
        'normal_avg_pl': normal_avg_pl,
        'hourly_toxic': hourly_toxic
    }

eur_tox = toxicity_analysis(eur_matched, 'EUR/USD') if len(eur_matched) > 0 else None
xau_tox = toxicity_analysis(xau_matched, 'XAU/USD') if len(xau_matched) > 0 else None

# Create summary table
if eur_tox and xau_tox:
    toxicity_summary = pd.DataFrame({
        'Metric': ['Total Matched Pairs', 'Toxic Trades (≤1 min)', 'Toxicity %', 
                   'Toxic Volume', 'Toxic Volume %', 'Toxic P/L', 'Normal P/L', 
                   'Avg P/L - Toxic', 'Avg P/L - Normal'],
        'EUR/USD': [
            f"{eur_tox['total_matched']:,}",
            f"{eur_tox['toxic_count']:,}",
            f"{eur_tox['toxic_pct']:.2f}%",
            f"{eur_tox['toxic_volume']:,.0f}",
            f"{eur_tox['toxic_volume_pct']:.2f}%",
            f"${eur_tox['toxic_pl']:,.2f}",
            f"${eur_tox['normal_pl']:,.2f}",
            f"${eur_tox['toxic_avg_pl']:,.2f}",
            f"${eur_tox['normal_avg_pl']:,.2f}"
        ],
        'XAU/USD': [
            f"{xau_tox['total_matched']:,}",
            f"{xau_tox['toxic_count']:,}",
            f"{xau_tox['toxic_pct']:.2f}%",
            f"{xau_tox['toxic_volume']:,.0f}",
            f"{xau_tox['toxic_volume_pct']:.2f}%",
            f"${xau_tox['toxic_pl']:,.2f}",
            f"${xau_tox['normal_pl']:,.2f}",
            f"${xau_tox['toxic_avg_pl']:,.2f}",
            f"${xau_tox['normal_avg_pl']:,.2f}"
        ],
        'Overall': [
            f"{eur_tox['total_matched'] + xau_tox['total_matched']:,}",
            f"{eur_tox['toxic_count'] + xau_tox['toxic_count']:,}",
            f"{(eur_tox['toxic_count'] + xau_tox['toxic_count'])/(eur_tox['total_matched'] + xau_tox['total_matched'])*100:.2f}%",
            '-',
            '-',
            f"${eur_tox['toxic_pl'] + xau_tox['toxic_pl']:,.2f}",
            f"${eur_tox['normal_pl'] + xau_tox['normal_pl']:,.2f}",
            '-',
            '-'
        ]
    })    
    all_toxic = pd.concat([
        eur_matched[eur_matched['Is_Toxic']].assign(Instrument='EUR/USD'),
        xau_matched[xau_matched['Is_Toxic']].assign(Instrument='XAU/USD')
    ])
    display(all_toxic[['Instrument', 'Entry_Time', 'Exit_Time', 'Holding_Time_Seconds', 
                       'Amount', 'Settled_PL']].head(10))
    


/tmp/ipykernel_31546/1835511392.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  toxic['Hour'] = toxic['Entry_Time'].dt.hour


,Instrument,Entry_Time,Exit_Time,Holding_Time_Seconds,Amount,Settled_PL
564,XAU/USD,2025-08-21 13:05:00,2025-08-21 13:05:00,0.0,100.0,-78.90
565,XAU/USD,2025-08-21 13:05:00,2025-08-21 13:05:00,0.0,50.0,-39.50
584,XAU/USD,2025-08-21 14:01:00,2025-08-21 14:01:00,0.0,50.0,-26.15
588,XAU/USD,2025-08-21 14:40:00,2025-08-21 14:40:00,0.0,100.0,-160.00
594,XAU/USD,2025-08-21 15:36:00,2025-08-21 15:36:00,0.0,100.0,59.90
664,XAU/USD,2025-08-22 14:20:00,2025-08-22 14:20:00,0.0,155.0,-2.55
665,XAU/USD,2025-08-22 14:20:00,2025-08-22 14:20:00,0.0,100.0,-149.00
961,XAU/USD,2025-09-01 09:02:00,2025-09-01 09:02:00,0.0,100.0,48.00
962,XAU/USD,2025-09-01 09:02:00,2025-09-01 09:02:00,0.0,100.0,49.00
963,XAU/USD,2025-09-01 09:02:00,2025-09-01 09:02:00,0.0,100.0,-39.00


<p>Position holding time – analysis of short, long term and averages on holding of the position in each instrument</p>

In [68]:
def holding_time(matched_df, instrument_name):
    if len(matched_df) == 0:
        print(f"No matched trades to analyze holding time for {instrument_name}.")
        return
    
    min_holding = matched_df['Holding_Time_Seconds'].min()
    max_holding = matched_df['Holding_Time_Seconds'].max()
    avg_holding = matched_df['Holding_Time_Seconds'].mean()
    median_holding = matched_df['Holding_Time_Seconds'].median()
    std_holding = matched_df['Holding_Time_Seconds'].std()
    q25 = matched_df['Holding_Time_Seconds'].quantile(0.25)
    q75 = matched_df['Holding_Time_Seconds'].quantile(0.75)

    print(f"Holding Time Analysis for {instrument_name}:")
    print(f"   Minimum Holding Time: {min_holding:.2f} seconds")
    print(f"   Maximum Holding Time: {max_holding:.2f} seconds")
    print(f"   Average Holding Time: {avg_holding:.2f} seconds")
    print(f"   Median Holding Time: {median_holding:.2f} seconds")
    print(f"   Standard Deviation: {std_holding:.2f} seconds")
    print(f"   25th Percentile: {q25:.2f} seconds")
    print(f"   75th Percentile: {q75:.2f} seconds")

    category_dist = matched_df['Hold_Category'].value_counts(normalize=True).sort_index() * 100
    category_counts = matched_df['Hold_Category'].value_counts().sort_index()
    for category in category_dist.index:
        count = category_counts[category]
        percent = category_dist[category]
        print(f"   {category}: {count:,} trades ({percent:.2f}%)")
    return{
        'Min Holding Time': min_holding,
        'Max Holding Time': max_holding,
        'Average Holding Time': avg_holding,
        'Median Holding Time': median_holding,
        'Std Deviation': std_holding,
        '25th Percentile': q25,
        '75th Percentile': q75
    }
eur_holding_stats = holding_time(eur_matched, 'EUR/USD') if len(eur_matched) > 0 else None
xau_holding_stats = holding_time(xau_matched, 'XAU/USD') if len(xau_matched) > 0 else None

Holding Time Analysis for EUR/USD:
   Minimum Holding Time: 60.00 seconds
   Maximum Holding Time: 667500.00 seconds
   Average Holding Time: 85242.79 seconds
   Median Holding Time: 26220.00 seconds
   Standard Deviation: 125817.52 seconds
   25th Percentile: 4305.00 seconds
   75th Percentile: 82830.00 seconds
   Toxic(≤1 min): 24 trades (1.08%)
   Ultra-short (1-5 min): 70 trades (3.14%)
   Short-Term (5-15 min): 216 trades (9.69%)
   Middle-Term (15-60 min): 175 trades (7.85%)
   Day (1-24 hrs): 1,227 trades (55.07%)
   Days(>24 hrs): 516 trades (23.16%)
Holding Time Analysis for XAU/USD:
   Minimum Holding Time: 0.00 seconds
   Maximum Holding Time: 1000140.00 seconds
   Average Holding Time: 59347.69 seconds
   Median Holding Time: 6840.00 seconds
   Standard Deviation: 151981.15 seconds
   25th Percentile: 1320.00 seconds
   75th Percentile: 42540.00 seconds
   Toxic(≤1 min): 213 trades (6.41%)
   Ultra-short (1-5 min): 247 trades (7.43%)
   Short-Term (5-15 min): 248 trades (7.

In [70]:
if eur_holding_stats and xau_holding_stats:
    stats_summary = pd.DataFrame({
        'Statistic': ['Min Hold Time (min)', 'Max Hold Time (days)', 'Average Hold (hrs)', 
                      'Median Hold (hrs)', 'Std Dev (hrs)', '25th Percentile (min)', '75th Percentile (hrs)'],
        'EUR/USD': [
            f"{eur_holding_stats['Min Holding Time']:.2f}",
            f"{eur_holding_stats['Max Holding Time']:.2f}",
            f"{eur_holding_stats['Average Holding Time']:.2f}",
            f"{eur_holding_stats['Median Holding Time']:.2f}",
            f"{eur_holding_stats['Std Deviation']:.2f}",
            f"{eur_holding_stats['25th Percentile']:.2f}",
            f"{eur_holding_stats['75th Percentile']:.2f}"
        ],
        'XAU/USD': [
            f"{xau_holding_stats['Min Holding Time']:.2f}",
            f"{xau_holding_stats['Max Holding Time']:.2f}",
            f"{xau_holding_stats['Average Holding Time']:.2f}",
            f"{xau_holding_stats['Median Holding Time']:.2f}",
            f"{xau_holding_stats['Std Deviation']:.2f}",
            f"{xau_holding_stats['25th Percentile']:.2f}",
            f"{xau_holding_stats['75th Percentile']:.2f}"
        ]
    })
    display(stats_summary)

,Statistic,EUR/USD,XAU/USD
0,Min Hold Time (min),60.00,0.00
1,Max Hold Time (days),667500.00,1000140.00
2,Average Hold (hrs),85242.79,59347.69
3,Median Hold (hrs),26220.00,6840.00
4,Std Dev (hrs),125817.52,151981.15
5,25th Percentile (min),4305.00,1320.00
6,75th Percentile (hrs),82830.00,42540.00


<h4> Visualizations </h4>

In [ ]:

fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('EUR/USD Cumulative Position', 'XAU/USD Cumulative Position'),
    vertical_spacing=0.12
)

# EUR/USD Position
fig.add_trace(
    go.Scatter(
        x=trade_eur['Transfer Date'],
        y=trade_eur['Cumulative Position'],
        mode='lines',
        name='EUR/USD Position',
        line=dict(color='blue', width=2),
        fill='tozeroy',
        fillcolor='rgba(0, 100, 255, 0.1)'
    ),
    row=1, col=1
)

# XAU/USD Position
fig.add_trace(
    go.Scatter(
        x=trade_xau['Transfer Date'],
        y=trade_xau['Cumulative Position'],
        mode='lines',
        name='XAU/USD Position',
        line=dict(color='gold', width=2),
        fill='tozeroy',
        fillcolor='rgba(255, 215, 0, 0.1)'
    ),
    row=2, col=1
)

fig.add_hline(y=0, line_dash="dash", line_color="gray", row=1, col=1)
fig.add_hline(y=0, line_dash="dash", line_color="gray", row=2, col=1)

fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_yaxes(title_text="Position", row=1, col=1)
fig.update_yaxes(title_text="Position", row=2, col=1)

fig.update_layout(
    height=800,
    title_text="Cumulative Position Over Time",
    showlegend=True,
    hovermode='x unified'
)

fig.show()



In [82]:
# Weekly Position Comparison (Bar Chart)
fig = go.Figure()

fig.add_trace(go.Bar(
    x=eur_weekly.index,
    y=eur_weekly['Cumulative Position'],
    name='EUR/USD',
    marker_color='blue'
))

fig.add_trace(go.Bar(
    x=xau_weekly.index,
    y=xau_weekly['Cumulative Position'],
    name='XAU/USD',
    marker_color='gold'
))

fig.update_layout(
    title='End-of-Week Position Comparison',
    xaxis_title='Week Number',
    yaxis_title='End Position',
    barmode='group',
    height=500
)

fig.show()


In [86]:

# Cumulative P/L Curve 
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=trade_eur['Transfer Date'],
    y=trade_eur['Cumulative_PL'],
    mode='lines',
    name='EUR/USD',
    line=dict(color='blue', width=2)
))

fig.add_trace(go.Scatter(
    x=trade_xau['Transfer Date'],
    y=trade_xau['Cumulative_PL'],
    mode='lines',
    name='XAU/USD',
    line=dict(color='gold', width=2)
))

# Combined
df_combined = time_trades.sort_values('Transfer Date').copy()
df_combined['Combined_Cumulative_PL'] = df_combined['Settled PL'].fillna(0).cumsum()

fig.add_trace(go.Scatter(
    x=df_combined['Transfer Date'],
    y=df_combined['Combined_Cumulative_PL'],
    mode='lines',
    name='Combined',
    line=dict(color='green', width=3, dash='dot')
))

fig.update_layout(
    title='Cumulative P/L (Equity Curve)',
    xaxis_title='Date',
    yaxis_title='Cumulative P/L ($)',
    hovermode='x unified',
    height=600
)

fig.show()


In [ ]:
# Daily P/L Bar Chart
fig = make_subplots(rows=2, cols=1, subplot_titles=('EUR/USD Daily P/L', 'XAU/USD Daily P/L'))

# EUR/USD
colors_eur = ['green' if x > 0 else 'red' for x in daily_eur_pl['Daily Settled PL']]
fig.add_trace(
    go.Bar(x=daily_eur_pl.index, y=daily_eur_pl['Daily Settled PL'], 
           marker_color=colors_eur, name='EUR/USD', showlegend=False),
    row=1, col=1
)

# XAU/USD
colors_xau = ['green' if x > 0 else 'red' for x in daily_xau_pl['Daily Settled PL']]
fig.add_trace(
    go.Bar(x=daily_xau_pl.index, y=daily_xau_pl['Daily Settled PL'], 
           marker_color=colors_xau, name='XAU/USD', showlegend=False),
    row=2, col=1
)

fig.update_layout(height=800, title_text="Daily P/L")
fig.update_yaxes(title_text="P/L ($)", row=1, col=1)
fig.update_yaxes(title_text="P/L ($)", row=2, col=1)

fig.show()



In [96]:
# Weekly P/L Waterfall Chart
fig = go.Figure(go.Waterfall(
    name="Weekly P/L",
    orientation="v",
    measure=["relative"] * len(weekly_pl_summary) + ["total"],
    x=[f"Week {i}" for i in weekly_pl_summary.index] + ["Total"],
    y=list(weekly_pl_summary['Total Weekly P/L']) + [weekly_pl_summary['Total Weekly P/L'].sum()],
    connector={"line": {"color": "rgb(63, 63, 63)"}},
))

fig.update_layout(
    title="Weekly P/L Waterfall",
    xaxis_title="Week",
    yaxis_title="P/L ($)",
    height=500
)

fig.show()

In [ ]:

# Scatter Plot - Trade Duration vs Volume
if len(eur_matched) > 0 and len(xau_matched) > 0:
    fig = go.Figure()
    
    # EUR/USD
    fig.add_trace(go.Scatter(
        x=eur_matched['Holding_Time_Seconds'],
        y=eur_matched['Amount'],
        mode='markers',
        name='EUR/USD',
        marker=dict(
            size=8,
            color=['red' if x else 'blue' for x in eur_matched['Is_Toxic']],
            opacity=0.6
        ),
        text=[f"P/L: ${pl:.2f}" for pl in eur_matched['Settled_PL']],
        hovertemplate='%{text}<br>Hold: %{x:.0f}s<br>Volume: %{y:,.0f}'
    ))
    
    fig.add_trace(go.Scatter(
        x=xau_matched['Holding_Time_Seconds'],
        y=xau_matched['Amount'],
        mode='markers',
        name='XAU/USD',
        marker=dict(
            size=8,
            color=['red' if x else 'gold' for x in xau_matched['Is_Toxic']],
            opacity=0.6,
            symbol='diamond'
        ),
        text=[f"P/L: ${pl:.2f}" for pl in xau_matched['Settled_PL']],
        hovertemplate='%{text}<br>Hold: %{x:.0f}s<br>Volume: %{y:,.0f}'
    ))
    
    # Toxicity threshold line
    fig.add_vline(x=60, line_dash="dash", line_color="red", 
                  annotation_text="1 Min Threshold", annotation_position="top")
    
    fig.update_layout(
        title='Trade Duration vs Volume (Red = Toxic ≤1 min)',
        xaxis_title='Holding Time (seconds)',
        yaxis_title='Trade Volume',
        xaxis_type='log',
        height=600
    )
    
    fig.show()

In [98]:
# oxicity Proportion (Donut Chart)
if eur_tox and xau_tox:
    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{'type':'pie'}, {'type':'pie'}]],
        subplot_titles=('EUR/USD Toxicity', 'XAU/USD Toxicity')
    )
    
    # EUR/USD
    fig.add_trace(
        go.Pie(
            labels=['Toxic (≤1 min)', 'Normal (>1 min)'],
            values=[eur_tox['toxic_count'], eur_tox['total_matched'] - eur_tox['toxic_count']],
            hole=0.4,
            marker_colors=['red', 'green'],
            textinfo='percent+label'
        ),
        row=1, col=1
    )
    
    # XAU/USD
    fig.add_trace(
        go.Pie(
            labels=['Toxic (≤1 min)', 'Normal (>1 min)'],
            values=[xau_tox['toxic_count'], xau_tox['total_matched'] - xau_tox['toxic_count']],
            hole=0.4,
            marker_colors=['red', 'green'],
            textinfo='percent+label'
        ),
        row=1, col=2
    )
    
    fig.update_layout(title_text="Toxicity Proportion", height=500)
    fig.show()


In [101]:
#Box Plot - Holding Time Comparison
if len(eur_matched) > 0 and len(xau_matched) > 0:
    fig = go.Figure()
    
    fig.add_trace(go.Box(
        y=eur_matched['Holding_Time_Hours'],
        name='EUR/USD',
        marker_color='blue',
        boxmean='sd'
    ))
    
    fig.add_trace(go.Box(
        y=xau_matched['Holding_Time_Hours'],
        name='XAU/USD',
        marker_color='gold',
        boxmean='sd'
    ))
    
    fig.update_layout(
        title='Holding Time Distribution Comparison',
        yaxis_title='Holding Time (hours)',
        yaxis_type='log',
        height=600
    )
    
    fig.show()

In [104]:
# Holding Time vs P/L Scatter
if len(eur_matched) > 0 and len(xau_matched) > 0:
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=eur_matched['Holding_Time_Hours'],
        y=eur_matched['Settled_PL'],
        mode='markers',
        name='EUR/USD',
        marker=dict(size=6, color='blue', opacity=0.5)
    ))
    
    fig.add_trace(go.Scatter(
        x=xau_matched['Holding_Time_Hours'],
        y=xau_matched['Settled_PL'],
        mode='markers',
        name='XAU/USD',
        marker=dict(size=6, color='gold', opacity=0.5, symbol='diamond')
    ))
    
    fig.update_layout(
        title='Holding Time vs P/L Correlation',
        xaxis_title='Holding Time (hours)',
        yaxis_title='P/L ($)',
        xaxis_type='log',
        height=600
    )
    
    fig.add_hline(y=0, line_dash="dash", line_color="gray")
    
    fig.show()